In [1]:
################## 시험 안내 문구 및 코드 ##################
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("../dataset/hr-data/X_test.csv")
X_train = pd.read_csv("../dataset/hr-data/X_train.csv")
y_train = pd.read_csv("../dataset/hr-data/y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': pred}).to_csv('003000000.csv', index=False)

---
## My Code

---
## Solution

## EDA

In [2]:
# 데이터 크기 확인
X_train.shape, y_train.shape, X_test.shape

((19158, 13), (19158, 2), (2126, 13))

In [3]:
# 데이터 샘플 확인
X_train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8


In [4]:
# 예측 해야할 타겟 샘플 확인 (0과 1이지만 문제에서는 확률값으로 예측하라고 되어 있음)
y_train.head()

,enrollee_id,target
0,8949,1
1,29725,0
2,11561,0
3,33241,1
4,666,0


In [5]:
# 예측할 데이터 균형 확인
y_train['target'].value_counts()

0    14381
1     4777
Name: target, dtype: int64

In [6]:
# 데이터 타입 확인
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.9+ MB
None


In [7]:
# 결측치 확인
print("##Train##")
print(X_train.isnull().sum())
print("\n##Test##")
print(X_test.isnull().sum())

##Train##
enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
dtype: int64

##Test##
enrollee_id                 0
city                        0
city_development_index      0
gender                    510
relevent_experience         0
enrolled_university        31
education_level            52
major_discipline          310
experience                  5
company_size              621
company_type              634
last_new_job               40
training_hours              0
dtype: int64


In [8]:
# 이상치IQR
# training_hours
import numpy as np
Q1 = np.percentile(X_train['training_hours'],25)
Q3 = np.percentile(X_train['training_hours'],75)
IQR = Q3 - Q1
outdata1 = X_train[X_train['training_hours']<(Q1 - 1.5 * IQR)]
outdata2 = X_train[X_train['training_hours']>(Q3 + 1.5 * IQR)]
len(outdata1), len(outdata2)
# 삭제하긴 많음

(0, 984)

In [9]:
# city_development_index       
Q1 = np.percentile(X_train['city_development_index'],25)
Q3 = np.percentile(X_train['city_development_index'],75)
IQR = Q3 - Q1
outdata1 = X_train[X_train['city_development_index']<(Q1 - 1.5 * IQR)]
outdata2 = X_train[X_train['city_development_index']>(Q3 + 1.5 * IQR)]
len(outdata1), len(outdata2)
#삭제 예정

(17, 0)

In [10]:
# 데이터 확인
obj_cols = np.array(X_train.columns[X_train.dtypes == object])
for col in obj_cols:
    print("\n##### : ",col)
    print(X_train[col].value_counts())
obj_cols


##### :  city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64

##### :  gender
Male      13221
Female     1238
Other       191
Name: gender, dtype: int64

##### :  relevent_experience
Has relevent experience    13792
No relevent experience      5366
Name: relevent_experience, dtype: int64

##### :  enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

##### :  education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

##### :  major_discipline
STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, d

array(['city', 'gender', 'relevent_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience',
       'company_size', 'company_type', 'last_new_job'], dtype=object)

## Preprocessing

In [11]:
# 심플 결측치 처리
X_train = X_train.fillna("X")
X_test = X_test.fillna("X")

In [12]:
# 이상치 삭제
print(X_train.shape)
ind = X_train[X_train['city_development_index']<(Q1 - 1.5 * IQR)].index
X_train = X_train.drop(index=ind, axis=0)
y_train = y_train.drop(index=ind, axis=0)

print(X_train.shape)

(19158, 13)
(19141, 13)


## 피처 엔지니어링

In [13]:
from sklearn.preprocessing import LabelEncoder

all_df = pd.concat([X_train.assign(ind="train"), X_test.assign(ind="test")])
le = LabelEncoder()
all_df[obj_cols] = all_df[obj_cols].apply(le.fit_transform)

X_train = all_df[all_df['ind'] == 'train']
X_train = X_train.drop('ind',axis=1)
X_train

X_test = all_df[all_df['ind'] == 'test']
X_test = X_test.drop('ind',axis=1)
X_test

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,78,0.827,1,0,0,0,5,19,8,6,0,21
1,9858,5,0.920,0,0,3,0,5,15,9,5,0,98
2,31806,64,0.624,1,1,3,1,6,20,9,5,6,15
3,27385,26,0.827,1,0,3,2,5,2,0,5,0,39
4,27724,5,0.920,1,0,3,0,5,21,3,5,4,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,27163,55,0.878,1,1,3,2,5,7,8,5,4,58
2122,24507,116,0.698,3,1,3,0,5,12,9,6,6,133
2123,21339,64,0.624,0,1,0,1,6,13,9,6,6,56
2124,1289,5,0.920,1,1,3,0,2,7,9,4,3,15


In [14]:
# 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

n_cols = ['city_development_index', 'training_hours']
X_train[n_cols] = scaler.fit_transform(X_train[n_cols])
X_test[n_cols] = scaler.transform(X_test[n_cols])
X_train

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,8949,5,0.094444,1,0,3,0,5,21,9,6,0,-0.169231
1,29725,77,-0.705556,1,1,3,0,5,6,5,5,4,0.000000
2,11561,64,-1.550000,3,1,0,0,5,15,9,6,6,0.553846
3,33241,14,-0.633333,3,1,2,0,1,20,9,5,6,0.076923
4,666,50,-0.755556,1,0,3,2,5,21,5,1,3,-0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,55,-0.138889,1,1,3,0,2,5,9,6,0,-0.076923
19154,31398,5,0.094444,1,0,3,0,5,5,9,6,3,0.076923
19155,24576,5,0.094444,1,0,3,0,5,21,5,5,3,-0.046154
19156,5756,94,-0.561111,1,0,3,1,6,20,6,5,1,0.769231


## 모델 학습 및 평가

In [15]:
# 학습용 데이터와 검증용 데이터로 구분
# 학습에 많은 데이터를 사용하기 위해 검증은 작게 가져감
from sklearn.model_selection import train_test_split


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['target'], test_size=0.1, random_state=2022)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((17226, 13), (1915, 13), (17226,), (1915,))

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
rf = RandomForestClassifier(random_state = 2022)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)[:,1]
print(roc_auc_score(y_val, pred))

0.8031070768759241


In [17]:
# Test데이터
pred = rf.predict_proba(X_test)[:,1]

In [18]:
# csv생성코드
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': pred}).to_csv('hr-data.csv', index=False)

## 채점

In [19]:
# 체점
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score

with open( "../dataset/hr-data/answer.pickle", "rb" ) as file:
    ans = pickle.load(file)
    ans = pd.DataFrame(ans)
print(roc_auc_score(ans['target'], pred))

0.77509520609319
